In [1]:
import torch
torch.__version__on__

'1.6.0'

In [2]:
import torch.nn.functional as F

In [3]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 3)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # Affice operation
        self.fc1 = torch.nn.Linear(16*6*6, 120) # 6x6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [6]:
params = list(net.parameters())
print(len(params))

10


In [7]:
print(params[0].size())

torch.Size([6, 1, 3, 3])


Try a random 32x32 input

In [9]:
inp = torch.randn(1,1,32,32)
out = net(inp)
out

tensor([[ 0.0842,  0.0505, -0.0303, -0.0021,  0.0807, -0.0448,  0.1069, -0.0011,
          0.0791,  0.0253]], grad_fn=<AddmmBackward>)

Zero the gradient buffers for all parameters and backprops with random gradients

In [10]:
net.zero_grad()
out.backward(torch.randn(1,10))

# Loss function

In [11]:
out = net(inp)
target = torch.randn(10)
target

tensor([-0.6015, -0.7801, -1.1632,  0.4622,  1.8093, -0.6923,  1.6578, -0.3096,
        -0.5397, -0.0032])

In [12]:
out

tensor([[ 0.0842,  0.0505, -0.0303, -0.0021,  0.0807, -0.0448,  0.1069, -0.0011,
          0.0791,  0.0253]], grad_fn=<AddmmBackward>)

In [13]:
target = target.view(1, -1)
target

tensor([[-0.6015, -0.7801, -1.1632,  0.4622,  1.8093, -0.6923,  1.6578, -0.3096,
         -0.5397, -0.0032]])

In [14]:
criterion = torch.nn.MSELoss()

In [15]:
loss = criterion(out, target)
print(loss)

tensor(0.8950, grad_fn=<MseLossBackward>)


```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```

In [17]:
loss.grad_fn

In [18]:
loss.grad_fn.next_functions[0][0] # linear

In [19]:
loss.grad_fn.next_functions[0][0].next_functions[0][0] # ReLU

# Backprop

In [20]:
net.zero_grad()

In [21]:
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [22]:
loss.backward()

In [23]:
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([-0.0070, -0.0077, -0.0045,  0.0089,  0.0095,  0.0065])


```python
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
```

In [25]:
import torch.optim as optim

In [26]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

In [28]:
optimizer.zero_grad()
out = net(inp)
loss = criterion(out, target)
loss.backward()
optimizer.step()